In [27]:
import time
import sys

import tweepy
from tweepy import Cursor

In [28]:
consumer_key="Gzq1IKIPRpKfEmhc2czHrfIAH"
consumer_secret="7lXDKTQsRGx9jG1r8MJpUdwHCUROI34lgwR6GDq3vXokWotiCs"
access_token="717556130-IPSrD4HIJuu48ZqIgWirDFlZHyVTK572kmjjrXTK"
access_token_secret="TIZI5p09wX1dZxehca0rvuW97wdgsw2SAVX4bbjLFe9bP"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [29]:
#get neo4j set up
#note, you have to have neo4j running and on the default port
from py2neo import Graph, authenticate

authenticate("hobby-janfgbjnjildgbkedbobfpol.dbs.graphenedb.com:24789", "mytweets", "DaHSZJ8myG1AMZfNcvkh")
graph = Graph("http://hobby-janfgbjnjildgbkedbobfpol.dbs.graphenedb.com:24789/db/data/", bolt = False)

In [36]:
def create_or_get_node(graph, twitter_user,labels=[]):
    data = {'id_str': twitter_user.id_str,
        'name': twitter_user.name,
        'screen_name': twitter_user.screen_name,
        'description': twitter_user.description,
        'url': twitter_user.url,
        'followers_count': twitter_user.followers_count,
        'friends_count': twitter_user.friends_count,
        'listed_count': twitter_user.listed_count,
        'statuses_count': twitter_user.statuses_count,
        'favourites_count': twitter_user.favourites_count,
        'location': twitter_user.location,
        'time_zone': twitter_user.time_zone,
        'utc_offset': twitter_user.utc_offset,
        'lang': twitter_user.lang,
        'profile_image_url': twitter_user.profile_image_url,
        'geo_enabled': twitter_user.geo_enabled,
        'verified': twitter_user.verified,
        'notifications': twitter_user.notifications,
    }
    query_string = """
        MERGE (u:User {id_str:{id_str}}) 
        ON CREATE SET
"""+   (('u:'+',u:'.join(labels)+",") if labels else '') +"""
            u.name={name},
            u.screen_name={screen_name},
            u.description={description},
            u.url={url},
            u.followers_count={followers_count},
            u.friends_count={friends_count},
            u.listed_count={listed_count},
            u.statuses_count={statuses_count},
            u.favourites_count={favourites_count},
            u.location={location},
            u.time_zone={time_zone},
            u.utc_offset={utc_offset},
            u.lang={lang},
            u.profile_image_url={profile_image_url},
            u.geo_enabled={geo_enabled},
            u.verified={verified},
            u.notifications={notifications}
""" +   (("ON MATCH SET\n  u:"+',u:'.join(labels)) if labels else '') +"""
        RETURN u
    """
    n = graph.run(query_string, data)
    return n


In [38]:
twitter_user = api.get_user('neo4j')
create_or_get_node(graph,twitter_user)